## Part 2: Music generation with RNNs

We will generate music using Reccurrent Neural Network (RNN). We will train a model to learn the patterns in the raw sheet music in ABC Notation and then use this model to generate new music.

### 2.1 Dependecies

In [5]:
# Import Tensorflow 2.0
# %tensorflow_version 2.x
import tensorflow as tf 

# Download and import the MIT Introduction to Deep Learning package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
# assert len(tf.config.list_physical_devices('GPU')) > 0

2023-05-04 00:55:37.073070: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-04 00:55:37.129738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-04 00:55:37.129760: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


### 2.2 Dataset
We will use a dataset of thousand Irish folk songs collected by MIT, represented in the ABC notation.
We will start by donwloading the dataset and inspecting it.

In [6]:
# Download the dataset
songs = mdl.lab1.load_training_data()

# Print one of the somgs to inspect it in great detail.
example_song = songs[1]
print("\nExample song:")
print(example_song)

Found 817 songs in text

Example song:
X:2
T:An Buachaill Dreoite
Z: id:dc-hornpipe-2
M:C|
L:1/8
K:G Major
GF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!
DGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!
GA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!
B2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!


We can convert the ABC notation to an audio file and then play it back.

In [ ]:
# Convert the ABC notation to audio file and listen to it.
mdl.lab1.play_song(example_song)

One important think about the notation of the music does not simply contain information on the notes being played, but additionally there is meta information such as the somg title, ker, and tempo. 

How does the number of different characters that are present in the text file impact the complexity of learning problem.

This will become important soon, when we generate a numerical representation of the text data.

In [7]:
# Lets join the list of songs into a single string containing all songs
songs_joined = "\n\n".join(songs)

# Find all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")


There are 83 unique characters in the dataset


### 2.3 Process the dataset for the learning task
We are training a RNN model to learn pattern in ABC music, and then use this model to generate (i.e, predict) a new piece of music based on this learned information. 
Basically what we are doing is given a set of characters, what is the most probable next character? We'll train the model to perform this task. 
To achieve this, we will input a sequence of characters to the model,and train the model to predict the output, that is the following characters at each time step.
RNNs maintain an internal state that depends on previously seen elements and information about all characters seen up unitll a given moment will be taken into account in generating the prediction.

### Vectorize the text : Lookup table
Before training our RNN model, we will need to create a numerial representation of our text-based dataset. To do this, we will generate two lookup tables.
Lookup table1: Maps characters to numbers
Lookup table2: Maps numbers to characters

We can do this because all we care about is the unique characters present in the text.

In [13]:
### Define the numerical representation of text ###

# Create a mapping from characters to unique index.
# For example, to get the index of the character "d", 
# We can evaluate `char2idx["d"].
char2idx = {u:i for i, u in enumerate(vocab)}

# Create a mapping from indices to characters. This is
# the inverse from char2idx and allows us to convert back
# from unique index to the char in our vocanbulary. 
idx2char = np.array(vocab)

This gives us an integer representation for each character. Observe that the unique characters (i.e, our vocabulary) in the text are mapped as indices from 0 to `len(unique)`. 


In [24]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('{}: {},'.format(repr(char), char2idx[char]))
print(' .. \n}')

{
'\n': 0,
' ': 1,
'!': 2,
'"': 3,
'#': 4,
"'": 5,
'(': 6,
')': 7,
',': 8,
'-': 9,
'.': 10,
'/': 11,
'0': 12,
'1': 13,
'2': 14,
'3': 15,
'4': 16,
'5': 17,
'6': 18,
'7': 19,
 .. 
}


In [56]:
### Vectorize the songs string ###
def vectorize_string(string):
    return np.array([char2idx[c] for c in string])

vectorized_songs = vectorize_string(songs_joined)

print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

'X:1\nT:Alex' ---- characters mapped to int ----> [49 22 13  0 45 22 26 67 60 79]
